In [2]:
import tensorflow as tf
import pickle
import time
import numpy as np
from sentence_transformers import SentenceTransformer, util
import pickle
from collections import namedtuple
from datetime import datetime
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import hnswlib
import subprocess
import torch
from getPassageFromPID import DocContentManager

In [20]:
sentence_transformer_model = SentenceTransformer('all-MiniLM-L6-v2')

with open('.\\combined_corpus_embeddings\\combined_corpus_embeddings', 'rb') as file:
    corpus_embeddings = pickle.load(file)
    corpus_embeddings = torch.tensor(corpus_embeddings)
d = DocContentManager()

C:\Users\asg41\AppData\Local\Temp\ipykernel_16628\2254235322.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  corpus_embeddings = torch.tensor(corpus_embeddings)


In [41]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Download the set of stopwords from NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Function to extract keywords from a single query
def extract_keywords(query):
    # Tokenize the query
    words = word_tokenize(query)
    # Convert to lower case
    words = [word.lower() for word in words]
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    keywords = [word for word in words if word not in stop_words and word.isalnum()]
    return keywords





[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asg41\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asg41\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
query = input("Enter your query: ")

query = extract_keywords(query)
query = " ".join(query)
print(query)

cpp_executable_path = '.\\bm25.exe' 
command = [cpp_executable_path, query]
process = subprocess.run(command, capture_output=True, text=True)

relavant_docs = []

if process.returncode == 0:
    output_lines = process.stdout.strip().split('\n')

    # Process each line of output
    for line in output_lines:
        pid = int(line)
        relavant_docs.append(pid)
        #print(pid)
        #print(d.fetchPassageContent(pid))
    #print(process.stderr)
else:
    print("Error:", process.stderr)

print(relavant_docs)

played starsky hutch
[3430141, 900397, 73658, 900401, 2753185]


In [45]:
query_embeddings = []
for i in relavant_docs:
    if i<7999999:
        query_embeddings.append(corpus_embeddings[i])
    
final_passages = []
hits = util.semantic_search(query_embeddings, corpus_embeddings)
for hit in hits[0]:
    final_passages.append(int(hit['corpus_id']))

for pid in final_passages:
    print(str(pid)+" : ")
    print(d.fetchPassageContent(pid))
    print()

3430141 : 
The title of this show sums up viewers' attitudes toward it (this was the lowest-rated Starsky and Hutch ever), as their flamboyant informant Huggy Bear goes into business on his own with... See full summary Â».

900400 : 
Their radio handle was ZEBRA-3 and their street car was an eye catching tomato red, 1975-76 Ford Gran Torino automobile with a wide white stripe.

73661 : 
Paul Michael Glaser: Starsky and Hutch star at peace now after rage and guilt over family's HIV. 20 years ago he thought his life was over after his teacher wife and daughter died from Aids and his son was left infected by HIV. Thank you for subscribing!

900405 : 
The scene where Ben Stiller and Owen Wilson are wearing their shoulder holsters and the little towels came straight from a Starsky and Hutch (1975) poster of Paul Michael Glaser and David Soul doing the exact same thing.

3430138 : 
The stars of American cop show Starsky and Hutch are coming to Birmingham - along with their iconic car. David 

In [9]:
print(d.fetchPassageContent(7130335))

Augmented reality. Augmented reality (AR) is a live direct or indirect view of a physical, real-world environment whose elements are augmented by computer-generated or extracted real-world sensory input such as sound, video, graphics or GPS data.


In [19]:
print(d.fetchPassageContent(3923587))

Page 6 of 28. aicpa.org/FRC. 1. Accounts Receivable Subledger Standards. The AR subledger standard format is intended to accommodate basic analysis of the AR balance. This may. include analysis of the levels of activity during a specified period, how much of the AR balance is liquidated.


In [43]:
from sklearn.neighbors import NearestNeighbors
nn_model = NearestNeighbors(n_neighbors=100, metric='euclidean')
nn_model.fit(corpus_embeddings)

NearestNeighbors(metric='euclidean', n_neighbors=100)

In [46]:
distances, indices = nn_model.kneighbors(query_embeddings)
#most_similar_documents = [corpus_texts[i] for i in indices[0]]
indice = indices[0]


In [47]:
for pid in indice:
    print(str(pid)+" : ")
    print(d.fetchPassageContent(pid))
    print()

3430141 : 
The title of this show sums up viewers' attitudes toward it (this was the lowest-rated Starsky and Hutch ever), as their flamboyant informant Huggy Bear goes into business on his own with... See full summary Â».

900400 : 
Their radio handle was ZEBRA-3 and their street car was an eye catching tomato red, 1975-76 Ford Gran Torino automobile with a wide white stripe.

73661 : 
Paul Michael Glaser: Starsky and Hutch star at peace now after rage and guilt over family's HIV. 20 years ago he thought his life was over after his teacher wife and daughter died from Aids and his son was left infected by HIV. Thank you for subscribing!

900405 : 
The scene where Ben Stiller and Owen Wilson are wearing their shoulder holsters and the little towels came straight from a Starsky and Hutch (1975) poster of Paul Michael Glaser and David Soul doing the exact same thing.

3430138 : 
The stars of American cop show Starsky and Hutch are coming to Birmingham - along with their iconic car. David 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asg41\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asg41\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
